In [ ]:
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import InputExample
from sentence_transformers import losses
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

word_embedding_model = models.Transformer('distilroberta-base')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
data=pd.read_csv("/content/train_data.csv")
data.head()

,FileName,Keywords,query,pos,neg
0,81945,"쟁점, 금액, 중개, 수수, 객관적, 양도, 차익, 계산, 경비, 인정, 지급, 주...",쟁점금액을 중개수수료로 지급하였다고 하는 주장을 뒷받침할만한 객관적인 증빙이 없으므...,쟁점금액을 중개수수료로 지급하였다고 하는 주장을 뒷받침할만한 객관적인 증빙이 없으므...,쟁점금액을 중개수수료로 지급하였다고 하는 주장을 뒷받침할만한 객관적인 증빙이 없으므...


In [ ]:
# 아래 코드 설명 : 현재 위 샘플데이터는 row 개수가 1개 이지만 만약 row 개수가 20만개 이상일때 가정해서 말씀드리겠습니다.

# data.head() 를 써서 각 칼럼에 대해서 먼저 분석을 하자면 pos는 query와 유사하다고 우리(인간)가 판단하고 라벨링 한데이터

# neg는 그 반대입니다.

# 이 Train code는 triplet loss를 활용하며 Contrastive Learning 기반입니다.

# query 와 유사한거랑만 cosine similar가 1이 되게끔 하는 train만 실시할경우 negative pair 에 약해질수 있으니 triplet loss를 활용합니다.

# 여기서 100 은 row 수 입니다. 아래 코드에는 반영이 정확히 안되어있지만 실제 20만데이터 활용시 row 수로 변경합니다.

# 위 csv 파일의 경우 query 가 중복되는 row가 있을 수도 있지만 pos와 neg는 최소한 하나는 다른 row 여야 합니다.

train_data=[]

for i in range(100):

  dict={}

  dict["query"]=data.iloc[0,2] # i번째 행 2 번째 칼럼
  dict["pos"]=data.iloc[0,3]   # i번째 행 3 번째 칼럼
  dict["neg"]=data.iloc[0,4]   # i번째 행 4 번째 칼럼

  train_data.append(dict)

In [ ]:
# train code 에 fit 이 될수 있게끔 train_data 리스트를 train_examples 리스트로 process 합니다.

train_examples = []

for i in range(100):
  example = train_data[i]
  train_examples.append(InputExample(texts=[example['query'], example['pos'], example['neg']]))

In [ ]:
epochs = 100
batch_size=10

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
train_loss = losses.TripletLoss(model=model)

In [ ]:
# Train 시작

model.fit(train_objectives=[(train_dataloader, train_loss)],epochs=epochs)

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Version : 240330

In [ ]:
# 곧 업데이트 할 것들

# Inference code 작성

# Validation 도 할수 있게끔 Train code 수정

# Checkpoint 중간중간 훈련 도중 저장

# Checkpoint를 허깅페이스에 올리는 코드

# Checkpoint를 허깅페이스로부터 다시 가져오는 코드
